# Loading Pretrained Models

In this notebook, we experiment with different pretrained models like `VGG16` and `AlexNet`. For each model we will train it on a subset of the data and observe its performance on the corresponding validation set. From there, we will take the best performing model and finetune it further by training it on the entire dataset in `fine_tuning.ipynb`.

In [1]:
# Standard library imports
import json
import time

# PyTorch imports
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

# torchvision imports
from torchvision.datasets import ImageFolder
from torchvision.models import vgg16, VGG16_Weights, resnet50, ResNet50_Weights, alexnet, AlexNet_Weights

# Project-specific imports
from hamburger_hotdog_pizza_classifier.config import PROCESSED_DATA_DIR, MODELS_DIR, BATCH_SIZE, MOMENTUM, NUM_CLASSES, LEARNING_RATE
from utils import train_validate_model#, modify_model_output

# Logging and experiment tracking
from loguru import logger

# Load hyperparameters
batch_size = BATCH_SIZE
lr = LEARNING_RATE
momentum = MOMENTUM
num_classes = NUM_CLASSES

print('Imported necessary libraries/variables')

2024-07-26 12:51:21.954 | INFO     | hamburger_hotdog_pizza_classifier.config:<module>:10 - PROJ_ROOT path is: C:\Git\hamburger-hotdog-pizza-classifier


Imported necessary libraries/variables


In [2]:
# Directory paths
subset_image_path = PROCESSED_DATA_DIR / "pizza_hamburger_hotdog_20_percent"
train_dir = subset_image_path / 'train'
test_dir = subset_image_path / 'test'
valid_dir = subset_image_path / 'valid'

# Data loading parameters
weights = VGG16_Weights.DEFAULT
transform = weights.transforms()

# Loss function setup
criterion = nn.CrossEntropyLoss()

# Device setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Start with Pre-Trained Models

For this dataset, we will experiment with `Resnet50`, `VGG16`, and `AlexNet` 

Let's choose some popular CNN architectures as a starting point

First, we need to load these pretrained models

In [3]:
# alex_net = modify_model_output('alexnet', num_classes, device)
# vgg16 = modify_model_output('vgg16', num_classes, device)
# resnet50 = modify_model_output('resnet50', num_classes, device)

## 1. Trying ResNet50

In [8]:
# Set up weights and modify ResNet50 model
weights = ResNet50_Weights.DEFAULT
resnet50_model = resnet50(weights=weights)
resnet50_model = resnet50_model.to(device)

# Adjust the final fully connected layer to match the number of classes
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)

In [9]:
# Transform setup for ResNet50
resnet50_transform = weights.transforms()

# Data preparation
train_data_resnet = ImageFolder(train_dir, transform=resnet50_transform)
valid_data_resnet = ImageFolder(valid_dir, transform=resnet50_transform)

# Data loaders
train_loader_resnet = DataLoader(train_data_resnet, batch_size=batch_size, shuffle=True)
valid_loader_resnet = DataLoader(valid_data_resnet, batch_size=batch_size, shuffle=True)

In [10]:

device = next(resnet50_model.parameters()).device
print(f"The model is on device: {device}")

The model is on device: cuda:0


In [11]:
# Optimizer and save path setup
optimizer = optim.SGD(resnet50_model.parameters(), lr=lr, momentum=momentum)
model_save_path = MODELS_DIR / "slightly_fine_tuned" / "resnet50.pth"

# Logging setup
model_name = "ResNet50"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")

# Model training and validation
train_validate_model(
    num_epochs=10, 
    model=resnet50_model, 
    train_loader=train_loader_resnet, 
    valid_loader=valid_loader_resnet, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    model_save_path=model_save_path
)

Overall Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Images are on device: cuda:0
Labels are on device: cuda:0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

## 2. Trying VGG16

In [ ]:
# Set up weights and modify VGG16 model
weights = VGG16_Weights.DEFAULT
vgg16_model = vgg16(weights=weights)

# Adjust the final fully connected layer to match the number of classes
vgg16_model.classifier[6] = nn.Linear(vgg16_model.classifier[6].in_features, num_classes)

In [ ]:
# Transform setup for VGG16
vgg16_transform = weights.transforms()

# Data preparation
train_data_vgg16 = ImageFolder(train_dir, transform=vgg16_transform)
valid_data_vgg16 = ImageFolder(valid_dir, transform=vgg16_transform)

# Data loaders
train_loader_vgg16 = DataLoader(train_data_vgg16, batch_size=batch_size, shuffle=True)
valid_loader_vgg16 = DataLoader(valid_data_vgg16, batch_size=batch_size, shuffle=True)

In [ ]:
# Optimizer and save path setup
optimizer = optim.SGD(vgg16_model.parameters(), lr=lr, momentum=momentum)
model_save_path = MODELS_DIR / "slightly_fine_tuned" / "vgg16.pth"

# Logging setup
model_name = "VGG16"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")

# Model training and validation
train_validate_model(
    num_epochs=10, 
    model=vgg16_model, 
    train_loader=train_loader_vgg16, 
    valid_loader=valid_loader_vgg16, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    model_save_path=model_save_path
) 

## 3. Trying AlexNet

In [ ]:
# Set up weights and modify VGG16 model
weights = AlexNet_Weights.DEFAULT
alex_net_model = alexnet(weights=weights)

# Adjust the final fully connected layer to match the number of classes
alex_net_model.classifier[6] = nn.Linear(alex_net_model.classifier[6].in_features, num_classes)

In [ ]:
# Transform setup for AlexNet
alex_net_transform = weights.transforms()

# Data preparation
train_data_alex_net = ImageFolder(train_dir, transform=alex_net_transform)
valid_data_alex_net = ImageFolder(valid_dir, transform=alex_net_transform)

# Data loaders
train_loader_alex_net = DataLoader(train_data_alex_net, batch_size=batch_size, shuffle=True)
valid_loader_alex_net = DataLoader(valid_data_alex_net, batch_size=batch_size, shuffle=True)

In [ ]:
# Optimizer and save path setup
optimizer = optim.SGD(alex_net_model.parameters(), lr=lr, momentum=momentum)
model_save_path = MODELS_DIR / "slightly_fine_tuned" / "alex_net.pth"

# Logging setup
model_name = "AlexNet"
date_time = time.time()
logger.add(f"logs/{model_name}/training_log-{date_time}.log", format="{time} {level} {message}", level="INFO")

# Model training and validation
train_validate_model(
    num_epochs=10, 
    model=alex_net_model, 
    train_loader=train_loader_alex_net, 
    valid_loader=valid_loader_alex_net, 
    criterion=criterion, 
    optimizer=optimizer, 
    device=device, 
    model_save_path=model_save_path
) 

Since VGG16 was able to achieve nearly 88% accuracy on the validation set, let's start off with VGG16 as our base model to improve upon.